In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
import tkinter as tk
from tkinter import ttk, messagebox
from dotenv import load_dotenv
import os

warnings.filterwarnings("ignore")
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('API_KEY')

auth_manager =SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)


def get_artist_audio_features(artist_name):
    
    # Search for the artist
    results = sp.search(q=f'artist:{artist_name}', type='artist')
    if len(results['artists']['items']) == 0:
        print("Artist not found.")
        return None, None, None

    artist = results['artists']['items'][0]
    artist_id = artist['id']
    genres = artist['genres']
    popularity = artist['popularity']

    # Get top tracks for the artist
    top_tracks = sp.artist_top_tracks(artist_id)

    track_ids = [track['id'] for track in top_tracks['tracks'][:10]]

    # Get audio features for the top tracks
    audio_features = sp.audio_features(track_ids)

    # Calculate average audio features
    feature_keys = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness','valence','tempo']
    averages = {key: np.mean([track[key] for track in audio_features]) for key in feature_keys}

    return averages, genres, popularity

def find_similar_artists(df, target_features, genre, explicit_preference, role):
    
    # Narrow down dataset to artists with desired genre and explicit preference
    genre_artists = df[(df['genres'].apply(lambda g: genre.lower() in g)) & (df['mode_explicit'] == explicit_preference)]
    
    if genre_artists.empty:
        print(f"No artists found in the genre: {genre} with explicit preference: {explicit_preference}")
        return None
    
    # Classify artists to diffeerent roles based on their popularity
    genre_artists['role'] = genre_artists['popularity'].apply(lambda x: 'Headliner' if x > 60 else 'Supporting Act' if x > 30 else 'Opening Act')

    # Features used to build the model
    feature_columns = ['avg_danceability', 'avg_energy', 'avg_loudness', 'avg_speechiness', 'avg_acousticness', 'avg_instrumentalness','avg_valence','avg_tempo']
    
    # Use KNN to find suggested artists for the different roles
    suggested_artists = {}
    
    for i in ['Opening Act', 'Supporting Act', 'Headliner']:
        if i.lower() != role.lower():   
            artists = genre_artists[genre_artists['role'] == i]
            
            X = artists[feature_columns]
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            target_features_scaled = scaler.transform([list(target_features.values())])
        
            nbrs = NearestNeighbors(n_neighbors=3).fit(X_scaled)
            distances, indices = nbrs.kneighbors(target_features_scaled)
            
            suggested_artists[i] = artists.iloc[indices[0]]
    
    return suggested_artists


def search_artists():
    artist_name = artist_name_var.get()
    genre_of_interest = genre_var.get()
    explicit_preference = explicit_var.get() == 'Yes'
    role_input = role_var.get()

    target_features, genres, popularity = get_artist_audio_features(artist_name)
    if target_features is None:
        return

    similar_artists = find_similar_artists(df_artist_stats, target_features, genre_of_interest, explicit_preference, role_input)

    if similar_artists:
        result_text.set("")
        for role, artists in similar_artists.items():
            result_text.set(result_text.get() + f"\n{role}:\n")
            for i, (index, row) in enumerate(artists.iterrows()):
                result_text.set(result_text.get() + f"{i+1}. {row['artist_name']}\n")

# Load artists dataset
df_artist_stats = pd.read_csv('artists_dataset.csv')

# Create the main window
root = tk.Tk()
root.title("Artist Recommender")

# Define variables
artist_name_var = tk.StringVar()
genre_var = tk.StringVar()
explicit_var = tk.StringVar()
role_var = tk.StringVar()
result_text = tk.StringVar()

# Create and place widgets
tk.Label(root, text="Artist Name:").grid(row=0, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=artist_name_var).grid(row=0, column=1, padx=10, pady=10)

tk.Label(root, text="Genre of Interest:").grid(row=1, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=genre_var).grid(row=1, column=1, padx=10, pady=10)

tk.Label(root, text="Explicit Content:").grid(row=2, column=0, padx=10, pady=10)
tk.OptionMenu(root, explicit_var, "Yes", "No").grid(row=2, column=1, padx=10, pady=10)

tk.Label(root, text="Role:").grid(row=3, column=0, padx=10, pady=10)
tk.OptionMenu(root, role_var, "Headliner", "Supporting Act", "Opening Act").grid(row=3, column=1, padx=10, pady=10)

tk.Button(root, text="Search", command=search_artists).grid(row=4, columnspan=2, padx=10, pady=10)

tk.Label(root, textvariable=result_text, justify='left').grid(row=5, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()